In [ ]:
import numpy as np
import pandas as pd
import psycopg2
from tqdm import tqdm
tqdm.pandas()

In [282]:
business_raw = pd.read_csv('csv_files/yelp_academic_dataset_business.csv')
business_raw.index +=1

In [ ]:
business_raw.head()

In [ ]:
business_raw.shape

In [ ]:
business_raw[business_raw["address"].isna()].shape

In [ ]:
business_raw[business_raw["attributes"].isna()].shape

In [ ]:
business_raw[business_raw["business_id"].isna()].shape

In [ ]:
business_raw[business_raw["categories"].isna()].shape

In [ ]:
business_raw[business_raw["city"].isna()]

In [ ]:
business_raw[business_raw["hours"].isna()].shape

In [ ]:
business_raw[business_raw["is_open"].isna()].shape

In [ ]:
business_raw[business_raw["latitude"].isna()].shape

In [ ]:
business_raw[business_raw["longitude"].isna()].shape

In [ ]:
business_raw[business_raw["name"].isna()].shape

In [ ]:
business_raw[business_raw["postal_code"].isna()].shape

In [ ]:
business_raw[business_raw["review_count"].isna()].shape

In [ ]:
business_raw[business_raw["stars"].isna()].shape

In [ ]:
business_raw[business_raw["state"].isna()].shape

In [ ]:
business_raw[business_raw["is_open"] == 1][business_raw["hours"].isna()]

In [ ]:
business_raw[business_raw["postal_code"].isna()][business_raw["address"].isna() == False]

We have 0 business without a state, 1 business without a city, 659 businesses without a postal_code, 7682 businesses without an address among which only 545 has no postal_code, so we have 114 businesses with an address but no postal_code

In [ ]:
eval(business_raw["attributes"][2])


# Parse businesses ids

In [283]:
business_ids = business_raw["business_id"].reset_index().set_index("business_id")["index"].to_dict()
del business_raw["business_id"]
business_table = business_raw.reset_index().rename(columns={'index':'id'})
business_table

,id,address,attributes,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,1,2818 E Camino Acequia Drive,{},"Golf, Active Life",Phoenix,NaN,0,33.522143,-112.018481,Arizona Biltmore Golf Club,85016,5,3.0,AZ
1,2,30 Eglinton Avenue W,"{'GoodForMeal': ""{'dessert': False, 'latenight...","Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605499,-79.652289,Emerald Chinese Restaurant,L5R 3E7,128,2.5,ON
2,3,"1210 8th Street SW, Unit 220","{'BusinessParking': ""{'garage': False, 'street...","Local Services, Professional Services, Compute...",Calgary,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",1,51.041771,-114.081109,Nucleus Information Service,T2R 1L3,5,2.0,AB
3,4,"6955 N Durango Dr, Ste 1116","{'BusinessParking': ""{'garage': False, 'street...","Beauty & Spas, Hair Salons",Las Vegas,"{'Monday': '8:30-20:0', 'Tuesday': '8:30-20:0'...",1,36.287312,-115.289540,Amazing Cuts,89149,116,4.5,NV
4,5,4 E University Dr,{},"American (Traditional), Restaurants",Tempe,NaN,0,33.422192,-111.939615,Ruby Tuesday,85281,9,2.5,AZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192604,192605,521 St. Catherine Street W,"{'Ambience': ""{'romantic': False, 'intimate': ...","Restaurants, Venezuelan, Latin American",Montréal,NaN,0,45.504597,-73.568456,Arepas,H3B 1A6,15,3.5,QC
192605,192606,244 Victoria Street,{},"Arts & Entertainment, Performing Arts",Toronto,NaN,1,43.655290,-79.379541,Ed Mirvish Theatre,M5B 1V8,58,4.0,ON
192606,192607,10210 South 50th Pl,NaN,"Education, Middle Schools & High Schools",Phoenix,"{'Monday': '7:30-17:30', 'Tuesday': '7:30-17:3...",1,33.354284,-111.975538,BASIS Ahwatukee,85044,3,2.5,AZ
192607,192608,"6670 S Tenaya Way, Ste 120, The Body Spa","{'BusinessParking': ""{'garage': False, 'street...","Beauty & Spas, Eyelash Service, Waxing, Esthet...",Las Vegas,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-19:0', '...",1,36.066952,-115.251008,Brows Redefined,89113,15,5.0,NV


# Parse Locations

In [284]:
locations = business_table[["id", "address", "city", "latitude", "longitude", "postal_code", "state"]]
del business_table["address"]
del business_table["city"]
del business_table["latitude"]
del business_table["longitude"]
del business_table["postal_code"]
del business_table["state"]
locations = locations.rename(columns={'id':'business_id'})
locations

,business_id,address,city,latitude,longitude,postal_code,state
0,1,2818 E Camino Acequia Drive,Phoenix,33.522143,-112.018481,85016,AZ
1,2,30 Eglinton Avenue W,Mississauga,43.605499,-79.652289,L5R 3E7,ON
2,3,"1210 8th Street SW, Unit 220",Calgary,51.041771,-114.081109,T2R 1L3,AB
3,4,"6955 N Durango Dr, Ste 1116",Las Vegas,36.287312,-115.289540,89149,NV
4,5,4 E University Dr,Tempe,33.422192,-111.939615,85281,AZ
...,...,...,...,...,...,...,...
192604,192605,521 St. Catherine Street W,Montréal,45.504597,-73.568456,H3B 1A6,QC
192605,192606,244 Victoria Street,Toronto,43.655290,-79.379541,M5B 1V8,ON
192606,192607,10210 South 50th Pl,Phoenix,33.354284,-111.975538,85044,AZ
192607,192608,"6670 S Tenaya Way, Ste 120, The Body Spa",Las Vegas,36.066952,-115.251008,89113,NV


## States

In [285]:
state_table = locations["state"].drop_duplicates()
state_table = state_table.sort_values().reset_index()
state_table.index += 1
del state_table["index"]
state_table = state_table.reset_index().rename(columns={"index":"id", "state": "name"})
state_table

,id,name
0,1,AB
1,2,AK
2,3,AL
3,4,AR
4,5,AZ
5,6,BAS
6,7,BC
7,8,CA
8,9,CON
9,10,CT


In [286]:
state_dict = state_table.set_index("name")["id"].to_dict()
locations["state_id"] = locations["state"].progress_apply(lambda s: state_dict[s])
del locations["state"]
locations

100%|██████████| 192609/192609 [00:00<00:00, 577326.66it/s]


,business_id,address,city,latitude,longitude,postal_code,state_id
0,1,2818 E Camino Acequia Drive,Phoenix,33.522143,-112.018481,85016,5
1,2,30 Eglinton Avenue W,Mississauga,43.605499,-79.652289,L5R 3E7,23
2,3,"1210 8th Street SW, Unit 220",Calgary,51.041771,-114.081109,T2R 1L3,1
3,4,"6955 N Durango Dr, Ste 1116",Las Vegas,36.287312,-115.289540,89149,20
4,5,4 E University Dr,Tempe,33.422192,-111.939615,85281,5
...,...,...,...,...,...,...,...
192604,192605,521 St. Catherine Street W,Montréal,45.504597,-73.568456,H3B 1A6,25
192605,192606,244 Victoria Street,Toronto,43.655290,-79.379541,M5B 1V8,23
192606,192607,10210 South 50th Pl,Phoenix,33.354284,-111.975538,85044,5
192607,192608,"6670 S Tenaya Way, Ste 120, The Body Spa",Las Vegas,36.066952,-115.251008,89113,20


## Cities

In [287]:
city_table = locations[["city", "state_id"]].drop_duplicates()
city_table = city_table.reset_index()
del city_table["index"]
city_table.index += 1
city_table = city_table.reset_index()
city_table = city_table.rename(columns={"index":"id", "city":"name"})
city_table

,id,name,state_id
0,1,Phoenix,5
1,2,Mississauga,23
2,3,Calgary,1
3,4,Las Vegas,20
4,5,Tempe,5
...,...,...,...
1253,1254,Pahrump,20
1254,1255,ETOBICOKE,23
1255,1256,Rockey View,1
1256,1257,Pittsburch,24


In [288]:
city_dict = city_table.set_index(["name", "state_id"])["id"].to_dict()

In [289]:
locations["city_id"] = locations[["city", "state_id"]].progress_apply(lambda row: city_dict[(row[0], row[1])], axis=1, raw=True)
del locations["state_id"]
del locations["city"]
locations

100%|██████████| 192609/192609 [00:08<00:00, 23230.99it/s]


,business_id,address,latitude,longitude,postal_code,city_id
0,1,2818 E Camino Acequia Drive,33.522143,-112.018481,85016,1
1,2,30 Eglinton Avenue W,43.605499,-79.652289,L5R 3E7,2
2,3,"1210 8th Street SW, Unit 220",51.041771,-114.081109,T2R 1L3,3
3,4,"6955 N Durango Dr, Ste 1116",36.287312,-115.289540,89149,4
4,5,4 E University Dr,33.422192,-111.939615,85281,5
...,...,...,...,...,...,...
192604,192605,521 St. Catherine Street W,45.504597,-73.568456,H3B 1A6,7
192605,192606,244 Victoria Street,43.655290,-79.379541,M5B 1V8,23
192606,192607,10210 South 50th Pl,33.354284,-111.975538,85044,1
192607,192608,"6670 S Tenaya Way, Ste 120, The Body Spa",36.066952,-115.251008,89113,4


## Postal codes

In [290]:
postal_codes_table = locations[["city_id", "postal_code"]].drop_duplicates()
postal_codes_table = postal_codes_table.reset_index()
del postal_codes_table["index"]
postal_codes_table.index += 1
postal_codes_table = postal_codes_table.reset_index().rename(columns={"index":"id"})
postal_codes_table

,id,city_id,postal_code
0,1,1,85016
1,2,2,L5R 3E7
2,3,3,T2R 1L3
3,4,4,89149
4,5,5,85281
...,...,...,...
20436,20437,164,44128
20437,20438,16,M1E 4P7
20438,20439,536,J7Y 3L8
20439,20440,7,H3H 1Y2


In [291]:
postal_code_dict = postal_codes_table.set_index(["postal_code", "city_id"])["id"].to_dict()

In [292]:
locations["postal_code_id"] = locations[["postal_code", "city_id"]].progress_apply(lambda row: postal_code_dict[(row[0], row[1])], axis=1, raw=True)
del locations["city_id"]
del locations["postal_code"]
locations

100%|██████████| 192609/192609 [00:08<00:00, 23009.72it/s]


,business_id,address,latitude,longitude,postal_code_id
0,1,2818 E Camino Acequia Drive,33.522143,-112.018481,1
1,2,30 Eglinton Avenue W,43.605499,-79.652289,2
2,3,"1210 8th Street SW, Unit 220",51.041771,-114.081109,3
3,4,"6955 N Durango Dr, Ste 1116",36.287312,-115.289540,4
4,5,4 E University Dr,33.422192,-111.939615,5
...,...,...,...,...,...
192604,192605,521 St. Catherine Street W,45.504597,-73.568456,5513
192605,192606,244 Victoria Street,43.655290,-79.379541,13246
192606,192607,10210 South 50th Pl,33.354284,-111.975538,205
192607,192608,"6670 S Tenaya Way, Ste 120, The Body Spa",36.066952,-115.251008,260


# Parse categories

In [293]:
business_categories_temp = business_table[["id", "categories"]][business_table["categories"].progress_map(lambda x: type(x) == str)]
business_categories_temp["categories"] = business_categories_temp["categories"].progress_map(lambda cats: list(map(str.strip, cats.split(","))))

business_categories_temp

100%|██████████| 192127/192127 [00:00<00:00, 296098.34it/s]


,id,categories
0,1,"[Golf, Active Life]"
1,2,"[Specialty Food, Restaurants, Dim Sum, Importe..."
2,3,"[Local Services, Professional Services, Comput..."
3,4,"[Beauty & Spas, Hair Salons]"
4,5,"[American (Traditional), Restaurants]"
...,...,...
192604,192605,"[Restaurants, Venezuelan, Latin American]"
192605,192606,"[Arts & Entertainment, Performing Arts]"
192606,192607,"[Education, Middle Schools & High Schools]"
192607,192608,"[Beauty & Spas, Eyelash Service, Waxing, Esthe..."


In [294]:
business_categories = (business_categories_temp['categories']
    .progress_apply(lambda x: pd.Series(x))
    .stack()
    .reset_index(level=1, drop=True)
    .to_frame('categories')
    .join(business_categories_temp[['id']], how='left'))
business_categories

100%|██████████| 192127/192127 [00:56<00:00, 3412.95it/s]


,categories,id
0,Golf,1
0,Active Life,1
1,Specialty Food,2
1,Restaurants,2
1,Dim Sum,2
...,...,...
192608,Trusts,192609
192608,& Probates,192609
192608,Lawyers,192609
192608,Legal Services,192609


In [295]:
categories = business_categories["categories"]
categories = categories.drop_duplicates().reset_index()
del categories["index"]
categories.index += 1
categories = categories.reset_index().rename(columns={"index":"id", "categories": "name"})
categories

,id,name
0,1,Golf
1,2,Active Life
2,3,Specialty Food
3,4,Restaurants
4,5,Dim Sum
...,...,...
1295,1296,Island Pub
1296,1297,Homeopathic
1297,1298,Medical Foot Care
1298,1299,Geneticists


In [296]:
categories_dict = categories.reset_index().set_index("name")["index"].to_dict()
business_categories["categorie_id"] = business_categories["categories"].progress_map(lambda cat: categories_dict[cat])
del business_categories["categories"]
del business_table["categories"]
business_categories = business_categories.rename(columns={"id":"business_id"})
business_categories

100%|██████████| 788359/788359 [00:01<00:00, 580858.01it/s]


,business_id,categorie_id
0,1,0
0,1,1
1,2,2
1,2,3
1,2,4
...,...,...
192608,192609,479
192608,192609,480
192608,192609,58
192608,192609,59


# Parse attributes

In [297]:
attributes = business_raw[business_raw["attributes"].map(lambda x: type(x)==str)]["attributes"].progress_apply(eval)
del business_table["attributes"]
attributes

100%|██████████| 163773/163773 [00:02<00:00, 60199.60it/s]


1                                                        {}
2         {'GoodForMeal': '{'dessert': False, 'latenight...
3         {'BusinessParking': '{'garage': False, 'street...
4         {'BusinessParking': '{'garage': False, 'street...
5                                                        {}
                                ...                        
192602                                                   {}
192604                                                   {}
192605    {'Ambience': '{'romantic': False, 'intimate': ...
192606                                                   {}
192608    {'BusinessParking': '{'garage': False, 'street...
Name: attributes, Length: 163773, dtype: object

## Noise level

In [298]:
noise_level_temp = attributes[attributes.map(lambda x: "NoiseLevel" in x)].map(lambda x: eval(x["NoiseLevel"]))
noise_level = noise_level_temp.drop_duplicates().reset_index()
del noise_level["index"]
noise_level.index += 1
noise_level = noise_level.reset_index().rename(columns={"index":"id", "attributes":"level"})
noise_level

,id,level
0,1,loud
1,2,average
2,3,quiet
3,4,very_loud
4,5,None


In [299]:
noise_level_dict = noise_level.set_index("level")["id"].to_dict()
business_table["noise_level_id"] = noise_level_temp.map(lambda x: noise_level_dict[x])
business_table

,id,hours,is_open,name,review_count,stars,noise_level_id
0,1,NaN,0,Arizona Biltmore Golf Club,5,3.0,NaN
1,2,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,Emerald Chinese Restaurant,128,2.5,NaN
2,3,"{'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...",1,Nucleus Information Service,5,2.0,1.0
3,4,"{'Monday': '8:30-20:0', 'Tuesday': '8:30-20:0'...",1,Amazing Cuts,116,4.5,NaN
4,5,NaN,0,Ruby Tuesday,9,2.5,NaN
...,...,...,...,...,...,...,...
192604,192605,NaN,0,Arepas,15,3.5,NaN
192605,192606,NaN,1,Ed Mirvish Theatre,58,4.0,2.0
192606,192607,"{'Monday': '7:30-17:30', 'Tuesday': '7:30-17:3...",1,BASIS Ahwatukee,3,2.5,NaN
192607,192608,"{'Monday': '0:0-0:0', 'Tuesday': '9:0-19:0', '...",1,Brows Redefined,15,5.0,NaN


## Music

In [300]:
music_temp = attributes[attributes.map(lambda x: "Music" in x)].map(lambda x: eval(x["Music"]))
music_temp = music_temp[music_temp.map(lambda x: x != None)]
music_temp = music_temp.progress_map(lambda m: [key for key in m.keys() if m[key]])
music_temp = music_temp[music_temp.map(lambda l: len(l) > 0)].reset_index().rename(columns={"index":"business_id", "attributes":"name"})
music_temp

100%|██████████| 5181/5181 [00:00<00:00, 338384.11it/s]


,business_id,name
0,94,[jukebox]
1,140,[dj]
2,164,"[background_music, video]"
3,207,[background_music]
4,255,[background_music]
...,...,...
2376,192154,[background_music]
2377,192182,[karaoke]
2378,192394,[background_music]
2379,192408,[jukebox]


In [301]:
music_business_relation_table = (music_temp['name']
    .progress_apply(lambda x: pd.Series(x))
    .stack()
    .reset_index(level=1, drop=True)
    .to_frame('name')
    .join(music_temp[['business_id']], how='left'))
music_business_relation_table

100%|██████████| 2381/2381 [00:00<00:00, 3575.90it/s]


,name,business_id
0,jukebox,94
1,dj,140
2,background_music,164
2,video,164
3,background_music,207
...,...,...
2376,background_music,192154
2377,karaoke,192182
2378,background_music,192394
2379,jukebox,192408


In [302]:
music_table = music_business_relation_table["name"].drop_duplicates().reset_index()
del music_table["index"]
music_table.index += 1
music_table = music_table.reset_index().rename(columns={"index":"id"})
music_table

,id,name
0,1,jukebox
1,2,dj
2,3,background_music
3,4,video
4,5,live
5,6,karaoke


In [303]:
music_dict = music_table.set_index("name")["id"].to_dict()
music_business_relation_table["music_id"] = music_business_relation_table["name"].map(lambda x: music_dict[x])
del music_business_relation_table["name"]
music_business_relation_table

,business_id,music_id
0,94,1
1,140,2
2,164,3
2,164,4
3,207,3
...,...,...
2376,192154,3
2377,192182,6
2378,192394,3
2379,192408,1


## Business parking 

In [304]:
parking_temp = attributes[attributes.map(lambda x: "BusinessParking" in x)].map(lambda x: eval(x["BusinessParking"]))
parking_temp = parking_temp[parking_temp.map(lambda x: x != None)]
parking_temp = parking_temp.progress_map(lambda m: [key for key in m.keys() if m[key]])
parking_temp = parking_temp[parking_temp.map(lambda l: len(l) > 0)].reset_index().rename(columns={"index":"business_id", "attributes":"name"})
parking_temp

100%|██████████| 101478/101478 [00:00<00:00, 420830.85it/s]


,business_id,name
0,2,[lot]
1,3,[street]
2,4,[lot]
3,12,[lot]
4,22,[lot]
...,...,...
61241,192594,[lot]
61242,192599,[garage]
61243,192600,[lot]
61244,192601,[street]


In [305]:
parking_business_relation_table = (parking_temp['name']
    .progress_apply(lambda x: pd.Series(x))
    .stack()
    .reset_index(level=1, drop=True)
    .to_frame('name')
    .join(parking_temp[['business_id']], how='left'))
parking_business_relation_table

100%|██████████| 61246/61246 [00:16<00:00, 3822.60it/s]


,name,business_id
0,lot,2
1,street,3
2,lot,4
3,lot,12
4,lot,22
...,...,...
61241,lot,192594
61242,garage,192599
61243,lot,192600
61244,street,192601


In [306]:
business_parking_table = parking_business_relation_table["name"].drop_duplicates().reset_index()
business_parking_table.index += 1
del business_parking_table["index"]
business_parking_table = business_parking_table.reset_index().rename(columns={"index":"id"})
business_parking_table

,id,name
0,1,lot
1,2,street
2,3,garage
3,4,valet
4,5,validated


In [307]:
parking_dict = business_parking_table.set_index("name")["id"].to_dict()
parking_business_relation_table["parking_id"] = parking_business_relation_table["name"].map(lambda x: parking_dict[x])
del parking_business_relation_table["name"]
parking_business_relation_table

,business_id,parking_id
0,2,1
1,3,2
2,4,1
3,12,1
4,22,1
...,...,...
61241,192594,1
61242,192599,3
61243,192600,1
61244,192601,2


## Ambience

In [308]:
ambience_temp = attributes[attributes.map(lambda x: "Ambience" in x)].map(lambda x: eval(x["Ambience"]))
ambience_temp = ambience_temp[ambience_temp.map(lambda x: x != None)]
ambience_temp = ambience_temp.progress_map(lambda m: [key for key in m.keys() if m[key]])
ambience_temp = ambience_temp[ambience_temp.map(lambda l: len(l) > 0)].reset_index().rename(columns={"index":"business_id", "attributes":"name"})
ambience_temp

100%|██████████| 47547/47547 [00:00<00:00, 353638.36it/s]


,business_id,name
0,2,[casual]
1,19,[casual]
2,32,[casual]
3,50,[casual]
4,59,[casual]
...,...,...
26701,192567,"[touristy, casual]"
26702,192571,[casual]
26703,192580,[casual]
26704,192593,[casual]


In [309]:
ambience_business_relation_table = (ambience_temp['name']
    .progress_apply(lambda x: pd.Series(x))
    .stack()
    .reset_index(level=1, drop=True)
    .to_frame('name')
    .join(ambience_temp[['business_id']], how='left'))
ambience_business_relation_table

100%|██████████| 26706/26706 [00:07<00:00, 3606.45it/s]


,name,business_id
0,casual,2
1,casual,19
2,casual,32
3,casual,50
4,casual,59
...,...,...
26701,casual,192567
26702,casual,192571
26703,casual,192580
26704,casual,192593


In [310]:
ambience_table = ambience_business_relation_table["name"].drop_duplicates().reset_index()
ambience_table.index += 1
del ambience_table["index"]
ambience_table = ambience_table.reset_index().rename(columns={"index":"id"})
ambience_table

,id,name
0,1,casual
1,2,hipster
2,3,trendy
3,4,romantic
4,5,intimate
5,6,classy
6,7,divey
7,8,touristy
8,9,upscale


In [311]:
ambience_dict = ambience_table.set_index("name")["id"].to_dict()
ambience_business_relation_table["ambience_id"] = ambience_business_relation_table["name"].map(lambda x: ambience_dict[x])
del ambience_business_relation_table["name"]
ambience_business_relation_table

,business_id,ambience_id
0,2,1
1,19,1
2,32,1
3,50,1
4,59,1
...,...,...
26701,192567,1
26702,192571,1
26703,192580,1
26704,192593,1


## Good for meal

In [312]:
good_for_meal_temp = attributes[attributes.map(lambda x: "GoodForMeal" in x)].map(lambda x: eval(x["GoodForMeal"]))
good_for_meal_temp = good_for_meal_temp[good_for_meal_temp.map(lambda x: x != None)]
good_for_meal_temp = good_for_meal_temp.progress_map(lambda m: [key for key in m.keys() if m[key]])
good_for_meal_temp = good_for_meal_temp[good_for_meal_temp.map(lambda l: len(l) > 0)].reset_index().rename(columns={"index":"business_id", "attributes":"name"})
good_for_meal_temp

100%|██████████| 29715/29715 [00:00<00:00, 309637.51it/s]


,business_id,name
0,2,"[lunch, dinner]"
1,19,"[lunch, dinner]"
2,32,"[lunch, dinner]"
3,59,"[dessert, lunch, dinner]"
4,64,"[lunch, brunch, breakfast]"
...,...,...
21251,192548,[dinner]
21252,192567,"[lunch, brunch]"
21253,192571,[lunch]
21254,192580,"[lunch, dinner]"


In [313]:
good_for_meal_business_relation_table = (good_for_meal_temp['name']
    .progress_apply(lambda x: pd.Series(x))
    .stack()
    .reset_index(level=1, drop=True)
    .to_frame('name')
    .join(good_for_meal_temp[['business_id']], how='left'))
good_for_meal_business_relation_table

100%|██████████| 21256/21256 [00:05<00:00, 3707.29it/s]


,name,business_id
0,lunch,2
0,dinner,2
1,lunch,19
1,dinner,19
2,lunch,32
...,...,...
21252,brunch,192567
21253,lunch,192571
21254,lunch,192580
21254,dinner,192580


In [314]:
good_for_meal_table = good_for_meal_business_relation_table["name"].drop_duplicates().reset_index()
good_for_meal_table.index += 1
del good_for_meal_table["index"]
good_for_meal_table = good_for_meal_table.reset_index().rename(columns={"index":"id"})
good_for_meal_table

,id,name
0,1,lunch
1,2,dinner
2,3,dessert
3,4,brunch
4,5,breakfast
5,6,latenight


In [315]:
good_for_meal_dict = good_for_meal_table.set_index("name")["id"].to_dict()
good_for_meal_business_relation_table["ambience_id"] = good_for_meal_business_relation_table["name"].map(lambda x: good_for_meal_dict[x])
del good_for_meal_business_relation_table["name"]
good_for_meal_business_relation_table

,business_id,ambience_id
0,2,1
0,2,2
1,19,1
1,19,2
2,32,1
...,...,...
21252,192567,4
21253,192571,1
21254,192580,1
21254,192580,2


## dietary restrictions

In [316]:
dietary_restrictions_temp = attributes[attributes.map(lambda x: "DietaryRestrictions" in x)].map(lambda x: eval(x["DietaryRestrictions"]))
dietary_restrictions_temp = dietary_restrictions_temp[dietary_restrictions_temp.map(lambda x: x != None)]
dietary_restrictions_temp = dietary_restrictions_temp.progress_map(lambda m: [key for key in m.keys() if m[key]])
dietary_restrictions_temp = dietary_restrictions_temp[dietary_restrictions_temp.map(lambda l: len(l) > 0)].reset_index().rename(columns={"index":"business_id", "attributes":"name"})
dietary_restrictions_temp

100%|██████████| 52/52 [00:00<00:00, 72532.03it/s]


,business_id,name
0,963,[gluten-free]
1,2091,[halal]
2,3777,[gluten-free]
3,10048,[vegan]
4,12861,[vegetarian]
5,13187,"[dairy-free, vegan, vegetarian]"
6,14766,"[gluten-free, vegan]"
7,16687,"[dairy-free, gluten-free, vegan, soy-free, veg..."
8,16942,"[gluten-free, vegetarian]"
9,22468,[gluten-free]


In [317]:
dietary_restrictions_business_relation_table = (dietary_restrictions_temp['name']
    .progress_apply(lambda x: pd.Series(x))
    .stack()
    .reset_index(level=1, drop=True)
    .to_frame('name')
    .join(dietary_restrictions_temp[['business_id']], how='left'))
dietary_restrictions_business_relation_table

100%|██████████| 52/52 [00:00<00:00, 3218.01it/s]


,name,business_id
0,gluten-free,963
1,halal,2091
2,gluten-free,3777
3,vegan,10048
4,vegetarian,12861
...,...,...
48,vegan,158889
49,vegan,161767
49,vegetarian,161767
50,gluten-free,177625


In [318]:
dietary_restrictions_table = dietary_restrictions_business_relation_table["name"].drop_duplicates().reset_index()
dietary_restrictions_table.index += 1
del dietary_restrictions_table["index"]
dietary_restrictions_table = dietary_restrictions_table.reset_index().rename(columns={"index":"id"})
dietary_restrictions_table

,id,name
0,1,gluten-free
1,2,halal
2,3,vegan
3,4,vegetarian
4,5,dairy-free
5,6,soy-free


In [319]:
dietary_restrictions_dict = dietary_restrictions_table.set_index("name")["id"].to_dict()
dietary_restrictions_business_relation_table["dietary_restrictions_id"] = dietary_restrictions_business_relation_table["name"].map(lambda x: dietary_restrictions_dict[x])
del dietary_restrictions_business_relation_table["name"]
dietary_restrictions_business_relation_table

,business_id,dietary_restrictions_id
0,963,1
1,2091,2
2,3777,1
3,10048,3
4,12861,4
...,...,...
48,158889,3
49,161767,3
49,161767,4
50,177625,1


# Parse schedule

In [320]:
schedule_temp = business_raw["hours"]
del business_table["hours"]
schedule_temp = schedule_temp[schedule_temp.map(lambda x: type(x) == str)].progress_map(eval)
schedule_temp

100%|██████████| 147779/147779 [00:05<00:00, 26152.42it/s]


2         {'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...
3         {'Monday': '9:0-17:0', 'Tuesday': '9:0-17:0', ...
4         {'Monday': '8:30-20:0', 'Tuesday': '8:30-20:0'...
6         {'Monday': '10:0-21:0', 'Tuesday': '10:0-21:0'...
7         {'Monday': '9:0-18:0', 'Tuesday': '9:0-18:0', ...
                                ...                        
192603    {'Monday': '8:0-17:0', 'Tuesday': '8:0-20:0', ...
192604    {'Monday': '7:0-17:0', 'Tuesday': '7:0-17:0', ...
192607    {'Monday': '7:30-17:30', 'Tuesday': '7:30-17:3...
192608    {'Monday': '0:0-0:0', 'Tuesday': '9:0-19:0', '...
192609    {'Monday': '8:0-18:0', 'Tuesday': '8:0-18:0', ...
Name: hours, Length: 147779, dtype: object

In [338]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
day_table = pd.DataFrame(days, columns=["name"])
day_table.index += 1
day_table = day_table.reset_index().rename(columns={"index": "id"})
day_dict = day_table.set_index("name")["id"].to_dict()
day_table

,id,name
0,1,Monday
1,2,Tuesday
2,3,Wednesday
3,4,Thursday
4,5,Friday
5,6,Saturday
6,7,Sunday


In [342]:
def schedules_on(day):
    temp = schedule_temp[schedule_temp.map(lambda m: day in m)].progress_map(lambda m: m[day]).reset_index().rename(columns={"index": "business_id"})
    temp["day_id"] = day_dict[day]
    temp["start_at"] = temp["hours"].map(lambda s: s.split("-")[0])
    temp["end_at"] = temp["hours"].map(lambda s: s.split("-")[1])
    del temp["hours"]
    return temp

100%|██████████| 91336/91336 [00:00<00:00, 609027.28it/s]


In [344]:
schedule_table = pd.concat([schedules_on(day) for day in days])
schedule_table

100%|██████████| 91336/91336 [00:00<00:00, 564161.36it/s]


,business_id,day_id,start_at,end_at
0,2,1,9:0,0:0
1,3,1,9:0,17:0
2,4,1,8:30,20:0
3,6,1,10:0,21:0
4,7,1,9:0,18:0
...,...,...,...,...
91331,192594,7,12:0,16:0
91332,192596,7,0:0,0:0
91333,192601,7,11:0,17:0
91334,192602,7,0:0,0:0


In [ ]:
business_raw[business_raw["city"] == "New York city"]

In [ ]:
locations.loc[119357]

In [ ]:
postal_codes_table[postal_codes_table["id"] == 119358]

In [ ]:
city_table[city_table["id"] == 119358]

In [ ]:
business_raw

In [ ]:
business_table

# Store results

In [347]:
def store_table(table, file_name):
    table.to_csv("generated/{}.csv".format(file_name), index=False)
store_table(state_table, "state")
store_table(city_table, "city")
store_table(postal_codes_table, "postal_code")
store_table(business_table, "business")
store_table(locations, "business_locations")
store_table(categories, "categorie")
store_table(business_categories, "business_categorie")
store_table(noise_level, "noise_level")
store_table(music_table, "music")
store_table(music_business_relation_table, "music_business_relation")
store_table(business_parking_table, "business_parking")
store_table(parking_business_relation_table, "parking_business_relation")
store_table(ambience_table, "ambience")
store_table(ambience_business_relation_table, "ambience_business_relation")
store_table(good_for_meal_table, "good_for_meal")
store_table(good_for_meal_business_relation_table, "good_for_meal_business_relation")
store_table(dietary_restrictions_table, "dietary_restrictions")
store_table(dietary_restrictions_business_relation_table, "dietary_restrictions_business_relation")
store_table(day_table, "day")
store_table(schedule_table, "schedule")